In [428]:
import pandas as pd
import re
#!pip install pycountry
import pycountry
from dateutil import parser
import numpy as np

In [429]:
df = pd.read_csv('nuforc_reports.csv')

In [430]:
# #convert durations to an interval in seconds
# df['duration_sec'] = df['duration'][~df['duration'].isnull()].apply(lambda x:x.split(' ')[0])

# # I will take first element as final choice for second with range (e.g. 3-5 seconds, then take 3)
# df['duration_sec_clean'] = df['duration_sec'][(~df['duration'].isnull())].apply(lambda x:x.split('-')[0])

# df['duration_sec_clean'][(df['duration'].str.contains('minutes')) & (~df['duration'].isnull())].apply(lambda x:x.split('>')[0])
# contains_digit = any(map(str.isdigit, df['duration'][~df['duration'].isnull()]))

### Data cleansing

I noticed about 3.6% of duration fields don't have any information. 
I will not necessarily remove rows with NaN in duration field but will categorize them as "unknown" instead.

Steps are following in converting date/time into a standard format, e.g. ISO 8106
* Convert date_time from ISO8106 format into clearer format for better data processing.

Steps are following in creating new columns for converting durations to an interval in seconds:
* Create new column called "duration_int_flag" which indicates whether strings in duration field had any integer in it.
* Create new column called "duration_type" which indicates whether strings in duration field had any characters such as "sec"/"min"/"hour" (all lower case) to identify if duration had any one of these characters. All the durations that are not either second/minute/hour will be classified as "unknown".
* Create new column called "duration_figure" which extracts only integer values from "duration" field. I will select the first element from the list of each "duration_figure" as the final option (e.g, "5 to 10 seconds" will be classified as 5 seconds). Of course, we may miss some second/minute/hour that may have exceptional patterns (e.g. two to three hours, 40minutes to 1 hour) but I will do my best to keep the most relevent ones as much as I can as it is nearly impossible to detect every single pattern given the time I have. And in fact, majority of information in duration seem to be written with integer rather than with specific English letters such as "One"/"Two" and etc.
* Create new column called "duration_len" which calculates the length of each "duration_figure" to qualify fields with "duration_len" != 0. This is extra step to filter out any data anomalities happened during parsing process.

Steps are following in adding a country field and store the country in ISO 3166-1
* For simplicity, I will add CA/US/OTHERS (UK, Iran and etc) only.

For simplicty, I will drop rows that are NaN in "duration"/"date_time"/"shape"/"posted". Of course, one can impute NaN values by parsing out information from "stats" but for the time I have given, I will consider these as data errors and remove them.

In [431]:
df.head()

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12T18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582
1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22T18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.664800,-72.639300
2,I woke up late in the afternoon 3:30-4pm. I we...,NaN,NaN,NaN,NaN,NaN,Occurred : 4/1/2019 15:45 (Entered as : April...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,NaN,NaN
3,I was driving towards the intersection of fall...,Ottawa,ON,2019-04-17T02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18T00:00:00,45.381383,-75.708501
4,"In Peoria Arizona, I saw a cigar shaped craft ...",Peoria,NY,2009-03-15T18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18T00:00:00,NaN,NaN


In [432]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88125 entries, 0 to 88124
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   summary         88095 non-null  object 
 1   city            87891 non-null  object 
 2   state           82890 non-null  object 
 3   date_time       86938 non-null  object 
 4   shape           85627 non-null  object 
 5   duration        84954 non-null  object 
 6   stats           88088 non-null  object 
 7   report_link     88125 non-null  object 
 8   text            88070 non-null  object 
 9   posted          86938 non-null  object 
 10  city_latitude   72013 non-null  float64
 11  city_longitude  72013 non-null  float64
dtypes: float64(2), object(10)
memory usage: 8.1+ MB


Notice that we have total 88125 entries. We will remove rows with NaN in shape/duration/date_time to clean the data. The reason for deleting rows pertaining to these values is that without any of these values, one cannot generate clear-cut insight.
Of course, one can spend time on parsing "stats" to possibly fill up missing shape/duration/date_time values but we will skip this process as we may not have enough time to consider all the edge cases.

After cleaning the data, we have 82557 rows of information, about 6.3% of rows have been removed as a result.

In [535]:
df_clean

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,duration_int_flag,duration_type,duration_figure,duration_len,duration_figure_clean,duration_figure_sec,country
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12 18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22,37.343152,-77.408582,True,sec,[5],1.0,5.0,5.0,US
1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22 18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29,41.664800,-72.639300,True,sec,"[3, 5]",2.0,3.0,3.0,US
3,I was driving towards the intersection of fall...,Ottawa,ON,2019-04-17 02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18,45.381383,-75.708501,True,sec,[10],1.0,10.0,10.0,CA
4,"In Peoria Arizona, I saw a cigar shaped craft ...",Peoria,NY,2009-03-15 18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18,NaN,NaN,True,min,[2],1.0,2.0,120.0,US
5,"The object has flashing lights that are green,...",Kirbyville,TX,2019-04-02 20:25:00,disk,15 minutes,Occurred : 4/2/2019 20:25 (Entered as : 04/02...,http://www.nuforc.org/webreports/145/S145476.html,"The object has flashing lights that are green,...",2019-04-08,30.677200,-94.005200,True,min,[15],1.0,15.0,900.0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88120,4 lights in formation over Tempe appear while ...,Tempe,AZ,2019-10-02 20:00:00,formation,3 minutes,Occurred : 10/2/2019 20:00 (Entered as : 10/2...,http://www.nuforc.org/webreports/149/S149463.html,4 lights in formation over Tempe appear while ...,2019-10-04,33.414036,-111.920920,True,min,[3],1.0,3.0,180.0,US
88121,"2 bright star like lights in the NNW skys, ((...",Bolivar,MO,2019-10-02 20:00:00,light,20 seconds,Occurred : 10/2/2019 20:00 (Entered as : 10/0...,http://www.nuforc.org/webreports/149/S149405.html,2 bright star like lights in the NNW sky two b...,2019-10-04,37.642200,-93.399600,True,sec,[20],1.0,20.0,20.0,US
88122,I just witnessed a ‘Phoenix Lights’ type of fo...,North Port,FL,2019-10-02 20:03:00,formation,20 seconds,Occurred : 10/2/2019 20:03 (Entered as : 10/0...,http://www.nuforc.org/webreports/149/S149424.html,10/2/19 @ 8:03PM EST UFO SIGHTING in the 3428...,2019-10-04,27.076210,-82.223280,True,sec,[20],1.0,20.0,20.0,US
88123,"Witnessed an orange, slow moving light. Was lo...",Black Mountain,NC,2019-10-02 22:00:00,fireball,2 minutes,Occurred : 10/2/2019 22:00 (Entered as : 10/0...,http://www.nuforc.org/webreports/149/S149447.html,"Witnessed an orange, slow moving light. Was lo...",2019-10-04,35.605000,-82.313200,True,min,[2],1.0,2.0,120.0,US


In [433]:
df_clean = df[(~df['shape'].isnull()) & (~df['date_time'].isnull()) & (~df['duration'].isnull())]

In [434]:
(len(df) - len(df_clean)) / len(df) * 100  ### % of rows deleted

6.318297872340425

Let's use parser.parse function to convert date_time into proper datetime64[ns] format.

In [435]:
df_clean['date_time'] = df_clean['date_time'].apply(lambda x:parser.parse(x))

<ipython-input-435-8137b10857d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['date_time'] = df_clean['date_time'].apply(lambda x:parser.parse(x))


In [436]:
df_clean['posted'] = df_clean['posted'].apply(lambda x:parser.parse(x))

<ipython-input-436-7fe4bbff3646>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['posted'] = df_clean['posted'].apply(lambda x:parser.parse(x))


Let's create a function that indicates whether string contains integer or not. Eventually, we will create column that indicates "duration_int_flag" = TRUE/FALSE.

In [437]:
# create function that determines if there is any integer within string - if integer found, mark that as True. Ohterwise, False.
def numeric_finder(x):
    return any(c.isdigit() for c in x)

In [438]:
df_clean['duration_int_flag'] = df_clean['duration'][~df_clean['duration'].isnull()].apply(lambda x:numeric_finder(x))

<ipython-input-438-d13768c53d29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_int_flag'] = df_clean['duration'][~df_clean['duration'].isnull()].apply(lambda x:numeric_finder(x))


Let's set second/minute for each duration. I noticed there are durations specifically written as "X seconds/X minutes". Let's seperate these two.

In [439]:
df_clean['duration_type'] = 'unknown'
df_clean['duration_type'][(df_clean['duration_int_flag'] == True) & (df_clean['duration'].str.lower().str.contains('min'))] = 'min'
df_clean['duration_type'][(df_clean['duration_int_flag'] == True) & (df_clean['duration'].str.lower().str.contains('sec'))] = 'sec'
df_clean['duration_type'][(df_clean['duration_int_flag'] == True) & (df_clean['duration'].str.lower().str.contains('hour'))] = 'hour'

<ipython-input-439-2a667b82a495>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_type'] = 'unknown'
<ipython-input-439-2a667b82a495>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_type'][(df_clean['duration_int_flag'] == True) & (df_clean['duration'].str.lower().str.contains('min'))] = 'min'
C:\Users\andy\anaconda3\lib\site-packages\pandas\core\series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [440]:
# df['duration'][(df['duration'].str.contains('Second')) & (~df['duration'].isnull())].unique()

Let's extract integer values from rows that have duration_int_flag == True (rows that contain integer in string) and duration is not null (skip rows that does not have duration)

In [441]:
df_clean['duration_figure'] = df_clean['duration'][(df_clean['duration_int_flag'] == True)].apply(lambda x:re.findall(r'\d+', x))

<ipython-input-441-4d194b4c4e64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_figure'] = df_clean['duration'][(df_clean['duration_int_flag'] == True)].apply(lambda x:re.findall(r'\d+', x))


This is an extra step to qualify empty "duration_figure". Let's consider only non-empty "duration_figure".

In [442]:
df_clean['duration_len'] = df_clean['duration_figure'][~df_clean['duration_figure'].isnull()].apply(lambda x:len(x))

<ipython-input-442-d8eeb47e0a8a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_len'] = df_clean['duration_figure'][~df_clean['duration_figure'].isnull()].apply(lambda x:len(x))


In [443]:
df_clean['duration_figure'][df_clean['duration_len'] == 0] = np.nan
df_clean['duration_figure_clean'] = df_clean['duration_figure'][~df_clean['duration_figure'].isnull()].apply(lambda x:int(x[0]))

<ipython-input-443-57a81c1591fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_figure'][df_clean['duration_len'] == 0] = np.nan
C:\Users\andy\anaconda3\lib\site-packages\pandas\core\series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
<ipython-input-443-57a81c1591fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

Let's now calculate "duration_figure_sec" for each "duration_type"

In [444]:
df_clean['duration_figure_sec'] = df_clean['duration_figure_clean']
df_clean['duration_figure_sec'][df_clean['duration_type'] == 'min'] = df_clean['duration_figure_sec'][df_clean['duration_type'] == 'min'] * 60
df_clean['duration_figure_sec'][df_clean['duration_type'] == 'hour'] = df_clean['duration_figure_sec'][df_clean['duration_type'] == 'hour'] * 3600

<ipython-input-444-9fb86261bb32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_figure_sec'] = df_clean['duration_figure_clean']
<ipython-input-444-9fb86261bb32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['duration_figure_sec'][df_clean['duration_type'] == 'min'] = df_clean['duration_figure_sec'][df_clean['duration_type'] == 'min'] * 60
<ipython-input-444-9fb86261bb32>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

Let's find out how many states are from U.S. If not, it should be either Canada or other country. I imported state abbreviation list from github.

In [445]:
states_available = list(df_clean['state'].unique())
us_states_abbr = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')['Abbreviation'].tolist()

In [446]:
states_not_in_us = [i for i in states_available if i not in us_states_abbr]

I noticed except for NaN, all of them are from Canada. Let's create country column to indicate countries by state. I will follow ISO3166-1 Alpha 2 (https://en.wikipedia.org/wiki/ISO_3166-1). If it is not from U.S or Canada, for simplicity, I will label them as "Not US/CA".

In [447]:
df_clean['country'] = 'Not US/CA'
df_clean['country'][df_clean['state'].isin(states_not_in_us)] = 'CA'
df_clean['country'][~df_clean['state'].isin(states_not_in_us)] = 'US'
df_clean['country'][df_clean['state'].isnull()] = 'Not US/CA'

<ipython-input-447-89e87ade8637>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['country'] = 'Not US/CA'
<ipython-input-447-89e87ade8637>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['country'][df_clean['state'].isin(states_not_in_us)] = 'CA'
<ipython-input-447-89e87ade8637>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['country'][~df_clean['st

In [448]:
(len(df) - len(df_final)) / len(df) * 100

16.080567375886528

In [449]:
#df_final[df_final['country'] == 'Not US/CA']

In [450]:
df_final = df_clean[df_clean['duration_type'] != 'unknown'].reset_index(drop=True)

In [524]:
us_pop = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv')
can_pop = pd.read_csv('canada_pop.csv')

In [536]:
us_state_abbr = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

In [525]:
can_pop.columns = ['population','city']

In [526]:
us_pop.columns = ['city','state','population','lat','lon']

In [527]:
us_pop['city'] = us_pop['city'].str.lower()
can_pop['city'] = can_pop['city'].str.lower()
df_final['city'] = df_final['city'].str.lower()

In [463]:
df_final.groupby(['country','shape']).size()

country  shape    
CA       changing       71
         chevron        27
         cigar          59
         circle        293
         cone            9
                      ... 
US       rectangle    1267
         sphere       4870
         teardrop      598
         triangle     6432
         unknown      4687
Length: 63, dtype: int64

In [537]:
# def state_converter(x):
    
#     length = x.upper().split(' ')
    
#     if len(length) < 2:
#         return length[0][:2]
#     else:
#         return length[0][0] + length[1][0]

# us_pop['state_clean'] = us_pop['state'].apply(lambda x:state_converter(x))

In [547]:
us_state_abbr.columns = ['state','abbr']
us_pop = us_pop.merge(us_state_abbr, on = 'state')
us_pop = us_pop[['city','abbr','population']]
us_pop.columns = ['city','state','population_us']

In [550]:
df_final.merge(us_pop, how = 'left', on = ['city','state'])

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,duration_int_flag,duration_type,duration_figure,duration_len,duration_figure_clean,duration_figure_sec,country,population_us
0,My wife was driving southeast on a fairly popu...,chester,VA,2019-12-12 18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22,37.343152,-77.408582,True,sec,[5],1.0,5.0,5.0,US,NaN
1,I think that I may caught a UFO on the NBC Nig...,rocky hill,CT,2019-03-22 18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29,41.664800,-72.639300,True,sec,"[3, 5]",2.0,3.0,3.0,US,NaN
2,I was driving towards the intersection of fall...,ottawa,ON,2019-04-17 02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18,45.381383,-75.708501,True,sec,[10],1.0,10.0,10.0,CA,NaN
3,"In Peoria Arizona, I saw a cigar shaped craft ...",peoria,NY,2009-03-15 18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18,NaN,NaN,True,min,[2],1.0,2.0,120.0,US,NaN
4,"The object has flashing lights that are green,...",kirbyville,TX,2019-04-02 20:25:00,disk,15 minutes,Occurred : 4/2/2019 20:25 (Entered as : 04/02...,http://www.nuforc.org/webreports/145/S145476.html,"The object has flashing lights that are green,...",2019-04-08,30.677200,-94.005200,True,min,[15],1.0,15.0,900.0,US,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73949,4 lights in formation over Tempe appear while ...,tempe,AZ,2019-10-02 20:00:00,formation,3 minutes,Occurred : 10/2/2019 20:00 (Entered as : 10/2...,http://www.nuforc.org/webreports/149/S149463.html,4 lights in formation over Tempe appear while ...,2019-10-04,33.414036,-111.920920,True,min,[3],1.0,3.0,180.0,US,168228.0
73950,"2 bright star like lights in the NNW skys, ((...",bolivar,MO,2019-10-02 20:00:00,light,20 seconds,Occurred : 10/2/2019 20:00 (Entered as : 10/0...,http://www.nuforc.org/webreports/149/S149405.html,2 bright star like lights in the NNW sky two b...,2019-10-04,37.642200,-93.399600,True,sec,[20],1.0,20.0,20.0,US,NaN
73951,I just witnessed a ‘Phoenix Lights’ type of fo...,north port,FL,2019-10-02 20:03:00,formation,20 seconds,Occurred : 10/2/2019 20:03 (Entered as : 10/0...,http://www.nuforc.org/webreports/149/S149424.html,10/2/19 @ 8:03PM EST UFO SIGHTING in the 3428...,2019-10-04,27.076210,-82.223280,True,sec,[20],1.0,20.0,20.0,US,59212.0
73952,"Witnessed an orange, slow moving light. Was lo...",black mountain,NC,2019-10-02 22:00:00,fireball,2 minutes,Occurred : 10/2/2019 22:00 (Entered as : 10/0...,http://www.nuforc.org/webreports/149/S149447.html,"Witnessed an orange, slow moving light. Was lo...",2019-10-04,35.605000,-82.313200,True,min,[2],1.0,2.0,120.0,US,NaN


In [422]:
df_final.to_csv('ufo_final.csv',index = False)

In [423]:
### By word/By shape/By duration - by city
### # of reports by city
### # of reports by city per population (add population for each city from external source)

### Calculate avg. duration by City and see if high avg. duration means higher # of reports per population
    ## if this is the case, we know that certain city has higher # of reports since UFOs appeared for longer time on average.
    ## Maybe create simple linear regression/XGboost regression to confirm whether duration has relationship with # of reports/is important feature to predict # of reports or not
    

In [427]:
can_pop

,2021,name
0,2600000,Toronto
1,1600000,Montreal
2,1019942,Calgary
3,812129,Ottawa
4,712391,Edmonton
...,...,...
395,6099,Drayton Valley
396,6061,Farnham
397,6055,The Pas
398,6049,Bromont
